In [23]:
#Agregate our data to one table
import pandas as pd
import numpy as np
from shutil import rmtree
from os import listdir, makedirs
from os.path import isfile, join, exists


def apple(daily):
        return np.sum(daily == 'a')
def pen(daily):
        return np.sum(daily == 'p')

def store_stats(sell, supply, inventory, sell_parser):
    supply.set_index('date', inplace=True)
    inventory.set_index('date', inplace=True)
    sell.sku_num = sell.sku_num.map(sell_parser)
    sells_daily = sell.groupby('date')['sku_num'].agg([apple, pen])
    changes_daily = supply.reindex_like(sells_daily).fillna(0) - sells_daily
    monthly_changes = changes_daily.groupby(lambda date: date[:-2]).agg(np.cumsum)
    extend_inventory = inventory.reindex_like(monthly_changes).shift(1).fillna(axis=0, method='ffill').fillna(0)
    store_daily_state = extend_inventory + monthly_changes
    stolen_monthly = store_daily_state.reindex_like(inventory) - inventory
    return store_daily_state, stolen_monthly, sells_daily.groupby(lambda date: date[:-2]).sum()

def agregate_stats(stats):
    return pd.concat(stats).groupby(['year', 'state']).sum()

input_dir = 'input'
output_dir = 'output'
sell_parser = lambda transaction: transaction[6]
all_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
stores = [f[:-8] for f in all_files if f[-8:] == "sell.csv"]
if not exists(output_dir):
    makedirs(output_dir)
else:
    rmtree(output_dir)
    makedirs(output_dir)

statistics = []

for store in stores:
    inventory = pd.read_csv(input_dir + '/' + store + 'inventory.csv')
    sold = pd.read_csv(input_dir + '/' + store + 'sell.csv')
    supply = pd.read_csv(input_dir + '/' + store + 'supply.csv')
    print ("Loading...")
    state, stolen, sells = store_stats(sold, supply, inventory, sell_parser)
    stolen.to_csv(output_dir + '/' + store + 'steal.csv')
    state.to_csv(output_dir + '/' + store + 'daily.csv')
    sells.index = stolen.index
    stats = sells.join(stolen, lsuffix='_sold', rsuffix='_stolen').reset_index()
    stats['state'] = store[:2]
    stats['test'] = store[:6]
    stats['year'] = stats.date.map(lambda date: date[:4])
    statistics.append(stats)

agregate_stats(statistics).to_csv(output_dir + '/states.csv')    
print('All okey')

Loading...
Loading...
Loading...
Loading...
Loading...
Loading...
Loading...
Loading...
Loading...
All okey


In [24]:
#Print our final table
st = pd.read_csv("output/states.csv")
st

,year,state,apple_sold,pen_sold,apple_stolen,pen_stolen
0,2006,MS,2152006,155633,418.0,461.0
1,2007,MS,2150384,154730,377.0,346.0
2,2008,MS,2163559,154597,383.0,382.0
3,2009,MS,2152502,155409,433.0,454.0
4,2010,MS,2149787,155523,418.0,441.0
5,2011,MS,2154860,154158,436.0,452.0
6,2012,MS,2160040,155798,381.0,421.0
7,2013,MS,2157901,154496,361.0,444.0
8,2014,MS,2153434,154687,433.0,441.0
9,2015,MS,2152497,153562,370.0,395.0
